In [2]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
import pandas as pd
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard

In [3]:
df = pd.read_csv ('/Users/abby/Documents/fa21-neural-networks/final_project/all-the-news-2-1.csv')

/Users/abby/Applications/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'date', 'year', 'month', 'day', 'author',
       'title', 'article', 'url', 'section', 'publication'],
      dtype='object')

In [31]:
df_fewer_cols = df[['article','publication']]

In [15]:
df_vox = df_fewer_cols[df_fewer_cols['publication'] == 'Vox'].article

In [20]:
df_vox

0          This post is part of Polyarchy, an independent...
8          Google I/O, the company's big developer confer...
10         Elizabeth Warren is giving people a new reason...
15         Nancy Pelosi is getting visions of reclaiming ...
16         The nonpartisan director of the federal Office...
                                 ...                        
2674053    The United States continues to record far and ...
2674054    Zoom, the videoconferencing app that’s dominat...
2674055    Governors in several of the hardest-hit states...
2674056    New York City’s Commission on Human Rights wil...
2674057    On Tuesday, the White House’s coronavirus task...
Name: article, Length: 47272, dtype: object

In [19]:
df_vox[0]

'This post is part of Polyarchy, an independent blog produced by the political reform program at New America, a Washington think tank devoted to developing new ideas and new voices. Imagine you are an otherwise healthy 30-something who starts feeling weird. You are sometimes short of breath. You get migraines. Your feet start to swell a little. But otherwise, everything seems fine. You go to the doctor. The doctor runs some tests. She tells you,\xa0It\'s probably nothing, but these could be signs of a coming heart attack. You push for more certainty, but the doctor tells you she\'s not sure. The human body is a complex system. You\'re young and otherwise pretty healthy. There could be plenty of other explanations for what you\'re feeling. But it is a little worrying. So just to be on the safe side, maybe you should reduce the stress in your life and eat a healthier diet. What would you do? If you\'re a sensible person, you\'d probably err on the side of precaution. Sure, it might be no

In [27]:
df.groupby(['publication','year']).count()

Unnamed: 0  Unnamed: 0.1   date  month    day  author  \
publication year                                                          
Axios       2016           7             7      7      7      7       7   
            2017       15659         15659  15659  15659  15659   15652   
            2018       16543         16543  16543  16543  16543   16542   
            2019       15605         15605  15605  15605  15605   15604   
            2020           1             1      1      1      1       1   
...                      ...           ...    ...    ...    ...     ...   
Wired       2017        4530          4530   4530   4530   4530    4530   
            2018        4157          4157   4157   4157   4157    4157   
            2019        4074          4074   4074   4074   4074    4073   
            2020        1103          1103   1103   1103   1103    1102   
            2019         269           269    269    269    269     269   

                  title  article    url  section  
publication year                                  
Axios       2016      7        7      7        0  
            2017  15659    15193  15659        0  
            2018  16543    16512  16543        0  
            2019  15605    15598  15605        0  
            2020      1        1      1        0  
...                 ...      ...    ...      ...  
Wired       2017   4530     4514   4530     4530  
            2018   4157     4145   4157     4126  
            2019   4074     4068   4074     4074  
            2020   1103     1101   1103     1103  
            2019    269      269    269      269  

[150 rows x 10 columns]

In [28]:
df[df['publication']=='Fox News'].groupby('year').count()

,Unnamed: 0,Unnamed: 0.1,date,month,day,author,title,article,url,section,publication
year,,,,,,,,,,,
2016,56,56,56,56,56,56,56,56,56,56,56
2017,2464,2464,2464,2464,2464,2464,2464,2464,2464,2464,2464
2018,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624


In [39]:
df_fox_news = df[df['publication'] == "Fox News"]
df_fox_news = df_fox_news[df_fox_news['year'] == 2018]
df_fox_news = df_fox_news[['article']]
df_fox_news.to_csv('fox_news.csv', index=False)

In [30]:
df[df['year'] == 2018].groupby('publication').count()

,Unnamed: 0,Unnamed: 0.1,date,year,month,day,author,title,article,url,section
publication,,,,,,,,,,,
Axios,16543,16543,16543,16543,16543,16543,16542,16543,16512,16543,0
Business Insider,2204,2204,2204,2204,2204,2204,2165,2204,2202,2204,0
Buzzfeed News,8194,8194,8194,8194,8194,8194,8194,8194,8174,8194,0
CNBC,42965,42965,42965,42965,42965,42965,404,42964,40685,42965,30927
CNN,29736,29736,29736,29736,29736,29736,28821,29736,29262,29736,29736
Economist,5942,5942,5942,5942,5942,5942,0,5942,5635,5942,5942
Fox News,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624,17624
Gizmodo,6590,6590,6590,6590,6590,6590,6590,6590,6590,6590,5593
Hyperallergic,3045,3045,3045,3045,3045,3045,3044,3045,3036,3045,0
